# APO Lab 5 Segmentacja obrazu

In [ ]:
#from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from matplotlib import pyplot as plt

!WAŻNE! Aby poniższy plik działał na Google Colab, należy wgrać plik 'lena_gray.bmp', '3shapes.bmp' oraz 'water_coins.jpg'. Z panelu po lewej stronie wybieramy ikonę folderu ('Files') a następnie 'Upload' i wybieramy zdjęcie (wcześniej ściągnięte na dysk twardy z UBI).

# Zadanie 1

In [ ]:
# Wczytanie obrazu pierwotnego
img = cv2.imread('lena_gray.bmp', cv2.IMREAD_GRAYSCALE)
#img = cv2.imread('page.png', cv2.IMREAD_GRAYSCALE)

#cv2_imshow(img)
#plt.figure(figsize=(10,10))
plt.imshow(img, cmap='gray')

## Zad 1. a) Segmentacja obrazu przez progowanie z ręcznie wyznaczonym progiem (thresholding)


Algorytm progowania realizuje prostą zależność gdzie wartości powyżej ustalonego progu przyjmują wartośc 1 (lub 255) a wartości poniżej wartość 0.

![alt text](https://github.com/knave88/APO/raw/main/progowanie.png)

W efekcie otrzymujemy binarny obraz.

In [ ]:
# ręczne ustawienie progu
myThresh = 127
# progowanie z powyższyą wartością progu
ret,img_th_binary = cv2.threshold(img,myThresh,255,cv2.THRESH_BINARY)

#cv2_imshow(img_th_binary)
#plt.figure(figsize=(10,10))
plt.imshow(img_th_binary, cmap='gray')

W programie zaliczenowym należy dać możliwość użytkownikowi interaktywnego doboru progu (najlepiej z aktywnym podglądem, czyli odświeżaniem obrazu po zmianie wartości progu). Przyład implementacji poniżej z zastosowaniem suwaka (slider) do przyjęcia wartości progu od użytkowanika.

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def fun_thresh(slider_threshold_value):
    print("Wybrana wastość progu: " + str(slider_threshold_value))

    # progowanie z powyższyą wartością progu
    ret,img_th_binary2 = cv2.threshold(img,slider_threshold_value,255,cv2.THRESH_BINARY)

    #cv2_imshow(img_th_binary2)
    #plt.figure(figsize=(10,10))
    plt.imshow(cv2.cvtColor(img_th_binary2, cv2.COLOR_BGR2RGB))

    return slider_threshold_value

#interact(fun_thresh, slider_threshold_value=58); # widgets.IntSlider(min=-10, max=30, step=1, value=10)
interact(fun_thresh, slider_threshold_value=widgets.IntSlider(min=0, max=255, step=1, value=127));

## Zad 1. b) Segmentacja obrazu przez progowanie adaptacyjne (adaptive thresholding)

Funkcja progowania adaptacyjnego wykonuje lokalne progowanie, dla każdego piksela jest wyznaczany próg na podstawie obszaru otaczającego analizowany piksel, zgodnie ze schematem okna przesuwnego.

W bibliotece openCV aby użyć funkcji cv2.adaptiveThreshold należy podać jako parametry wejściowe:
*   obraz wejściowy (img)
*   maksymalną wartość dla obrazu (255)
*   sposób liczenia progu (adaptive_method = cv2.ADAPTIVE_THRESH_MEAN_C) na podstawie wartości średniej, lub (ADAPTIVE_THRESH_GAUSSIAN_C) średniej ważonej
*   progowanie wprost (threshold_type = cv2.THRESH_BINARY), lub (THRESH_BINARY_INV) z inwersją
*   rozmiar okna poddawanego analizie, powinno mieć wartość nieprzystą (block_size = 11)
*   stała wartość odejmowana od średniej (5)

Poniżej przedstawiono dwie wersje liczenia progu.

In [ ]:

max_value = 255
adaptive_method = cv2.ADAPTIVE_THRESH_MEAN_C
threshold_type = cv2.THRESH_BINARY
block_size = 11

# progowanie z powyższyą wartością progu
img_th_adapt = cv2.adaptiveThreshold(img, max_value, adaptive_method, threshold_type, block_size, 5)

#cv2_imshow(img_th_binary)
frame = cv2.hconcat((img, img_th_adapt))
#plt.figure(figsize=(10,10))
plt.imshow(frame, cmap='gray')

In [ ]:

max_value = 255
adaptive_method = cv2.ADAPTIVE_THRESH_GAUSSIAN_C # średniej ważonej
threshold_type = cv2.THRESH_BINARY
block_size = 11

# progowanie z powyższyą wartością progu
img_th_adapt = cv2.adaptiveThreshold(img, max_value, adaptive_method, threshold_type, block_size, 5)

#cv2_imshow(img_th_binary)
frame = cv2.hconcat((img, img_th_adapt))
#plt.figure(figsize=(10,10))
plt.imshow(frame, cmap='gray')

W tym przypadku od użytkownika należy przyjąć wartość rozmiaru okna przetwarania. Uwaga, należy pamiętać, że rozmiar okna przetwarzania powinien być liczbą nieparzystą większą lub równą 3.

In [ ]:
def fun_thresh_adapt(slider_threshold_window):
    print("Wybrana wielkosc okna: " + str(slider_threshold_window))

    max_value = 255
    adaptive_method = cv2.ADAPTIVE_THRESH_MEAN_C
    threshold_type = cv2.THRESH_BINARY
    block_size = slider_threshold_window         # wartosc podawana intraktywnie

    # progowanie z powyższyą wartością progu
    img_th_adapt = cv2.adaptiveThreshold(img, max_value, adaptive_method, threshold_type, block_size, 5)

    #cv2_imshow(img_th_adapt)
    frame = cv2.hconcat((img, img_th_adapt))
    #cv2_imshow(frame)
    plt.imshow(frame, cmap='gray')

    return slider_threshold_window

interact(fun_thresh_adapt, slider_threshold_window=widgets.IntSlider(min=0, max=255, step=1, value=21));

## Zad 1. c) Segmentacja obrazu przez progowanie metodą Otsu (Otsu thresholding)

Jest to metoda progowania, oparta na histogramie. Metoda polega na minimalizacji sumy ważonej wariancji dwóch klas (tła i obiektów pierwszego planu), co jest tożsame z maksymalizacją wariancji międzyklasowej. Metodę tę można rozszerzyć do metody wieloprogowej jak również do progowania lokalnego.

Często stosuje się najpierw lekkie rozmycie obrazu a dopiero potem progowanie Otsu. Pozwala to m.in. na uniknięcie małych artefaktów.

Publikacja: Nobuyuki Otsu, A Threshold Selection Method from Gray-Level Histograms, „IEEE Transactions on Systems, Man, and Cybernetics”, 9 (1), 1979, s. 62–66, DOI: 10.1109/TSMC.1979.4310076

In [ ]:
# Otsu's thresholding
ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Otsu's thresholding after Gaussian filtering
blur = cv2.GaussianBlur(img,(5,5),0)
ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

frame = cv2.hconcat((th2, th3))
#cv2_imshow(frame)
plt.imshow(frame, cmap='gray')

# Zadanie 2
## Segmentacja metodą GrabCut

In [ ]:
img = cv2.imread('lena.bmp', cv2.IMREAD_COLOR)


mask = np.zeros(img.shape[:2], np.uint8)


backgroundModel = np.zeros((1, 65), np.float64)
foregroundModel = np.zeros((1, 65), np.float64)


rectangle = (45, 80, 350, 450) # docelowo to pobieramy od uzytkownika


cv2.grabCut(img, mask, rectangle,
            backgroundModel, foregroundModel,
            3, cv2.GC_INIT_WITH_RECT)


mask2 = np.where((mask == 2)|(mask == 0), 0, 1).astype('uint8')

image_segmented = img * mask2[:, :, np.newaxis]

plt.imshow(cv2.cvtColor(image_segmented, cv2.COLOR_BGR2RGB))

# Zadanie 3

## Segmentacja obrazu metodą wododziałową (watershed)

Dział wodny (wododział, watershed) to granica pomiędzy zlewiskami różnych rzek
lub zbiorników wodnych. Wododziały są zawsze umiejscowione na grzbietach funkcji wysokości terenu.
W analizie i przetwarzaniu obrazów pod pojęciem działu wodnego rozumie się operację morfologiczną na obszarze, którego centrum stanowi lokalne minimum w obrazie. Obszary o małej intensywności stanowią w obrazie lokalne „doliny”, zaś obszary o dużej intensywności – lokalne „wzniesienia” . Jako wysokość terenu przyjmuje się amplitudę gradientu intensywności pikseli lub samą
intensywność, za wododział zaś – grzbiety tych funkcji.

Użycie metody watershed wymaga najpierw wyznaczenia znaczników obszarów które jednoznacznie należą do tła oraz takich które jednoznacznie należą do obiektów. Z pomocą tych obrazów jest wykonywane rozdzielenie obiektów sklejonych, co przedstawiono poniżej.

In [ ]:
# Wczytanie obrazu do segmentacji
img2 = cv2.imread('water_coins.jpg', cv2.IMREAD_COLOR) # water_coins.jpg
#cv2_imshow(img2)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))

In [ ]:
# sprowadzenie obrazu do szaroodcieniowego (grayscale)
img_gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
# progowanie obrazu czyli wstępne wyznaczenie obiektów
ret2,thresh = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

#cv2_imshow(thresh)
plt.imshow(thresh, cmap='gray')

In [ ]:
# odszumianie obrazu przez operacje morfologiczne
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 1)

#cv2_imshow(opening)
plt.imshow(opening, cmap='gray')

In [ ]:
# wyznaczenie jednoznacznych obszarów tła
sure_bg = cv2.dilate(opening,kernel,iterations=1)

#cv2_imshow(sure_bg)
plt.imshow(sure_bg, cmap='gray')

In [ ]:
# wyznaczenie jednoznacznych obszarów obiektów

# krok pośredni: transformata odległościowa
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
print ("Obraz transformaty odległościwej (pseudokolor dla lepszej wizualizacji)")
#cv2_imshow(cv2.applyColorMap(np.uint8(dist_transform*10), cv2.COLORMAP_JET))
plt.imshow(cv2.applyColorMap(np.uint8(dist_transform*10), cv2.COLORMAP_JET))


In [ ]:
# określenie jednoznacznych obszarów obiektów przez progowanie obrazu transformaty odlegościowej
print ("Obraz jednoznacznych obszarów należących do obiektów")
ret, sure_fg = cv2.threshold(dist_transform,0.5*dist_transform.max(),255,0)
sure_fg = np.uint8(sure_fg)
#cv2_imshow(sure_fg)
plt.imshow(sure_fg, cmap='gray')

In [ ]:
# wyznaczenie obszarów "niepewnych"
# takich w których znajdują się krawędzie obiektu
# obraz uzyskujemy przez odjęcie obszarów jednoznacznych
unknown = cv2.subtract(sure_bg,sure_fg)

#cv2_imshow(unknown)
plt.imshow(unknown, cmap='gray')

In [ ]:
# etykietowanie obiektów
ret, markers = cv2.connectedComponents(sure_fg)
print("Znalezionych obiektów:  "+str(np.max(markers)))
#cv2_imshow(cv2.applyColorMap(np.uint8(markers*10), cv2.COLORMAP_JET))
plt.imshow(cv2.applyColorMap(np.uint8(markers*10), cv2.COLORMAP_JET))
#(pseudokolor dla lepszej wizualizacji)

In [ ]:
# dodanie wartości 1 do etykiet, tak aby tło miało wartość 1 a nie 0.
markers = markers+1

# oznaczenie obszarów "niepewnych" jako zero
markers[unknown==255] = 0

#cv2_imshow(markers)
#cv2_imshow(cv2.applyColorMap(np.uint8(markers*10), cv2.COLORMAP_JET)) #(pseudokolor dla lepszej wizualizacji)
plt.imshow(cv2.applyColorMap(np.uint8(markers*10), cv2.COLORMAP_JET))

In [ ]:
# aplikacj algorytmu wododziału (watershed) do orygnalnego kolorowego obrazu (img2),
# dodatkowy argument wymagany do wykonania tej operacji to obliczony powyżej obraz etykietowany
markers2 = cv2.watershed(img2, markers)

#cv2_imshow(cv2.applyColorMap(np.uint8(markers2*10), cv2.COLORMAP_JET)) #(pseudokolor dla lepszej wizualizacji)
plt.imshow(cv2.applyColorMap(np.uint8(markers2*10), cv2.COLORMAP_JET))

print("Znaleziono "+ str(np.max(markers2)) + " obiektów.")

# Wynik:
# Granice obiektów zaznaczone w obrazie wartością -1
# Obiekty etykietowane kolejnymi wartościami

In [ ]:
# wstawienie linii krawędzi obiektów do obrazu szaroodcieniowego
img_gray[markers2 == -1] = 255

#cv2_imshow(img_gray)
plt.imshow(img_gray)

In [ ]:
# wstawienie linii krawędzi obiektów do oryginalnego obrazu kolorowego
img2[markers2 == -1] = [255,0,0]

#cv2_imshow(img2)
plt.imshow(img2)

In [ ]:
# wizualzacja obiektow (labelling)
markers2_colored = cv2.applyColorMap(np.uint8(markers2*10), cv2.COLORMAP_JET)
img_blend = cv2.addWeighted(np.stack((img_gray,)*3, axis=-1),0.7,markers2_colored,0.5,-1)
plt.figure(figsize=(10,10))
plt.imshow(img_blend)

Wynik algorytmu watershed zawarty jest w zmiennej 'markers2'. Jest to obraz odpowiadający rozmiarem obrazowi orygnialnemu w którym oddzielne obiekty są etykietowane (każdy obiekt ma przypisaną jedną wartość) a granice obiektów zaznaczone są wartością -1.

In [ ]:
# powyższa procedura zamknięta w jednej funkcji
def my_watershed(image):
  img_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

  ret2,thresh = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
  kernel = np.ones((3,3),np.uint8)
  opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 1)

  sure_bg = cv2.dilate(opening,kernel,iterations=1)
  dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)

  ret, sure_fg = cv2.threshold(dist_transform,0.5*dist_transform.max(),255,0)
  sure_fg = np.uint8(sure_fg)

  unknown = cv2.subtract(sure_bg,sure_fg)
  ret, markers = cv2.connectedComponents(sure_fg)

  markers = markers+1
  markers[unknown==255] = 0

  markers2 = cv2.watershed(image, markers)

  img_gray[markers2 == -1] = 255
  image[markers2 == -1] = [255,0,0]

  print("Znaleziono "+ str(np.max(markers2)) + " obiektów.")

  # wizuaizacja: obraz oryginalny oraz wynik algorytmy watershed
  frame = cv2.hconcat(( image, cv2.applyColorMap(np.uint8(markers2*10), cv2.COLORMAP_JET) ))
  #cv2_imshow(frame)
  plt.imshow(frame)

  return markers2

In [ ]:
# Dodatkowe przykłady
out1 = my_watershed(cv2.imread('soczewica2.jpg', cv2.IMREAD_COLOR))

In [ ]:
out2 = my_watershed(cv2.imread('ryz1.jpg', cv2.IMREAD_COLOR))

Jak widać na powyższych przykładach algorytm watershed nie jest idealny. Obiekty które nachodzą na siebie w większym stopniu niż stykanie się są trudne do rozdzielenia tą metodą. Ponadto, klastry zawierające dużą liczbę obietków również stanowią problem. Algorytm mylnie rozpoznaje granice lub nie rozdziela obiektów na pojedyncze.

# Zadanie 4
## Metoda Inpainting

In [ ]:
img = cv2.imread('lena_vandalized.bmp')
plt.imshow(img)


In [ ]:
mask = cv2.imread('lena_mask3.bmp', 0)
plt.imshow(mask)

In [ ]:
# Inpaint
img_inpainted = cv2.inpaint(img, mask, 3, cv2.INPAINT_TELEA) # alternatywa: cv2.INPAINT_TELEA lub cv2.INPAINT_NS

frame = cv2.hconcat((img, img_inpainted))
plt.figure(figsize=(20,10))
plt.imshow(frame)

# Zadanie 5
## Kompresja metodą RLE


Algorytm kompresji realizuje prostą formułę gdzie kodowane są ciągi identycznych symboli. Para zawiera symbol i liczbę jego powtórzeń.

![alt text](https://github.com/knave88/APO/raw/main/rle.jpg)


In [ ]:
def RLE_string_encode(input_string):
    count = 1
    prev = ''
    lst = []
    for character in input_string:
        if character != prev:
            if prev:
                entry = (prev,count)
                lst.append(entry)
                #print lst
            count = 1
            prev = character
        else:
            count += 1
    else:
        try:
            entry = (character,count)
            lst.append(entry)
            return (lst, 0)
        except Exception as e:
            print("Exception encountered {e}".format(e=e))
            return (e, 1)


In [ ]:
def RLE_string_decode(lst):
    q = ""
    for character, count in lst:
        q += character * count
    return q

In [ ]:
value = RLE_string_encode("aaaaapppppppppoooooooooooooo")
if value[1] == 0:
    print("Encoded value is {}".format(value[0]))
    RLE_string_decode(value[0])

# Zadanie 6
## Analiza obiektów

In [ ]:
# Wczytanie obrazu pierwotnego
img = cv2.imread('3shapes.bmp', cv2.IMREAD_GRAYSCALE)
#cv2_imshow(img)
plt.figure(figsize=(10,10))
plt.imshow(img, cmap='gray')

Funkcja cv2.findContours pozwala w prosty sposób wybrać z obrazu binarnego obiekty a właściwie ich kontury/obrysy. Jej ierwszym argumentem wejściowym jest oczywiście obraz. Dwa kolejne parametry definiują szczegóły generowania konturów. Drugi parametr określa czy znalezione zostaną wyłącznie kontóry zewnętrzne czy wszystie i określa hierarchiczność. W tym przykładzie będziemy korzystać z opcji *cv2.RETR_LIST*, która generuje wszystkie kontóry w obrazie bez ustalania ich hierarchii. Trzeci paramer określa czy i w jaki sposób kontury obiektów będą przybliżane (aproksymowane) aby zminimalizować liczbę punktów niezbędnych do przechowania. W tym przykładzie będziemy korzystać z opcji *cv2.CHAIN_APPROX_SIMPLE*, która enkoduje linie poziome, pionowe i nachylone pod kątem 45 stopni.

Pozostałe opcje drugiego parametru:


*   RETR_EXTERNAL
Python: cv.RETR_EXTERNAL
retrieves only the extreme outer contours. It sets hierarchy[i][2]=hierarchy[i][3]=-1 for all the contours.

*   RETR_LIST
Python: cv.RETR_LIST
retrieves all of the contours without establishing any hierarchical relationships.

*   RETR_CCOMP
Python: cv.RETR_CCOMP
retrieves all of the contours and organizes them into a two-level hierarchy. At the top level, there are external boundaries of the components. At the second level, there are boundaries of the holes. If there is another contour inside a hole of a connected component, it is still put at the top level.

*   RETR_TREE
Python: cv.RETR_TREE
retrieves all of the contours and reconstructs a full hierarchy of nested contours.

*   RETR_FLOODFILL
Python: cv.RETR_FLOODFILL

Pozostałe opcje trzeciego parametru:

*   CHAIN_APPROX_NONE
Python: cv.CHAIN_APPROX_NONE
stores absolutely all the contour points. That is, any 2 subsequent points (x1,y1) and (x2,y2) of the contour will be either horizontal, vertical or diagonal neighbors, that is, max(abs(x1-x2),abs(y2-y1))==1.

*   CHAIN_APPROX_SIMPLE
Python: cv.CHAIN_APPROX_SIMPLE
compresses horizontal, vertical, and diagonal segments and leaves only their end points. For example, an up-right rectangular contour is encoded with 4 points.

*   CHAIN_APPROX_TC89_L1
Python: cv.CHAIN_APPROX_TC89_L1
applies one of the flavors of the Teh-Chin chain approximation algorithm

*   CHAIN_APPROX_TC89_KCOS
Python: cv.CHAIN_APPROX_TC89_KCOS
applies one of the flavors of the Teh-Chin chain approximation algorithm


[Referencja] Dokumentacja: https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html#findcontours

In [ ]:
# preporcessing
ret,thresh = cv2.threshold(img,127,255,0) # pogowanie obrazu pierwotnego aby uzyskac obraz binarny
contours,hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) # funkcja znajdywania kontórow w obrazie binarnym
len(contours) # liczb znalezionych konturów

Jak łatwo zauważyć pomimo że na obrazie są trzy obiekty to algorytm znalazł 4 kontury. Zobaczmy dlaczego. Narysyjemy wszystkie znalezione kontury na obrazie.

In [ ]:
# Rysowanie kontórów na obrazie

img2 = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB) # konwersja z szaroodcieniowego do RGB (właściwie to kolejność BGR) aby można było rysować kolorowe kontury

# rysujemy pierwszy kontur kolorem niebieskim
cnt = contours[0]
cv2.drawContours(img2, [cnt], 0, (255,0,0), 3)

# rysujemy drugi kontur kolorem zielonym
cnt = contours[1]
cv2.drawContours(img2, [cnt], 0, (0,255,0), 3)

# rysujemy trzeci kontur kolorem czerwonym
cnt = contours[2]
cv2.drawContours(img2, [cnt], 0, (0,0,255), 3)

# rysujemy czwarty kontur innym kolorem
cnt = contours[3]
cv2.drawContours(img2, [cnt], 0, (255,255,0), 3)

#cv2_imshow(img2)
plt.figure(figsize=(10,10))
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))

Jak widać algorytm znajduje również kontur najbardziej zewnętrzny, czyli ramkę obrazu. Poniżej definicja funkcji która wczytuje obraz i wyświetla naniesione kolorowe kontury na obraz po progowaniu. Zachęcam do przetestowania różnych kombinacji parametrów funkcji *cv2.findContours*.

In [ ]:
def find_and_show_contours (fname):
  img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
  ret,thresh = cv2.threshold(img,127,255,0)
  contours,hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
# mode: cv2.RETR_EXTERNAL / cv2.RETR_FLOODFILL
# approximation: cv2.CHAIN_APPROX_NONE / cv2.CHAIN_APPROX_SIMPLE
  img3 = cv2.cvtColor(thresh,cv2.COLOR_GRAY2RGB)
  for cnt in contours:
    cv2.drawContours(img3, [cnt], 0, (random.randrange(50,200,25),random.randrange(50,200,25),random.randrange(50,200,25)), 3)
  #cv2_imshow(img3)
  plt.figure(figsize=(10,10))
  plt.imshow(img3)

In [ ]:
find_and_show_contours ('train_fasola.jpg')

## Zad 1. a) Obliczenie momentów

Do obliczania momentów centralnych obrazu (obiektu) służy wbudowana funkcja opencv *cv2.moments*. Może ona przyjmować jako argument wejściowy cały obraz lub kontur (uzyskany z poprzednio użytej funkcji *cv2.findContours*).

Funkcja ta wylicza momenty do 3 rzędu wiec w wyniku otrzymujemy 24 wartości. W języku python wynik jest w postaci 'dict'.

[Referencja] Dokumentacja: https://docs.opencv.org/2.4/modules/imgproc/doc/structural_analysis_and_shape_descriptors.html?highlight=cv2.moments#cv2.moments

In [ ]:
# obliczanie momentów dla całego obrazu
M_img = cv2.moments(img)
M_img

In [ ]:
# obliczanie momentów dla pojedynczego obiektu
cnt = contours[0]
M = cv2.moments(cnt)
M

## Zad 1. b) Obliczenie pola powierzchni i obwodu

In [ ]:
area = cv2.contourArea(cnt)
area

In [ ]:
perimeter = cv2.arcLength(cnt,True) # Drugi argument określa czy kształt ma zamknięty kontur (wtedy podajemy True) czy jest to krzywa
perimeter

## Zad 1. c) Obliczanie współczynników kształtu

In [ ]:
#aspect ratio
x,y,w,h = cv2.boundingRect(cnt)
aspect_ratio = float(w)/h
aspect_ratio

In [ ]:
#extent
area = cv2.contourArea(cnt)
x,y,w,h = cv2.boundingRect(cnt)
rect_area = w*h
extent = float(area)/rect_area
extent

In [ ]:
#solidity
area = cv2.contourArea(cnt)
hull = cv2.convexHull(cnt)
hull_area = cv2.contourArea(hull)
solidity = float(area)/hull_area
solidity

In [ ]:
# equivalentDiameter
area = cv2.contourArea(cnt)
equi_diameter = np.sqrt(4*area/np.pi)
equi_diameter